In [1]:
import os
import requests
import base64
from datetime import datetime, timedelta
from urllib.parse import urlencode
from pathlib import Path

In [2]:
auth_url = "https://accounts.spotify.com/api/token"
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_uri = os.getenv("REDIRECT_URI")
base_url = "https://api.spotify.com/v1" # base URL of all Spotify API endpoints
current_dir = Path.cwd()
home_dir = Path.home()
project_dir = current_dir.parent
scopes = [
    'user-follow-read',
    'user-library-read',
    'playlist-modify-private',
    'playlist-read-private',
    'user-top-read',
    'playlist-modify-public'
]

In [31]:
class SpotifyAPI():
    access_token = None
    access_token_expiry = datetime.now()
    client_id = None
    client_secret = None
    scope = 'user-top-read'
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_url_user = 'https://accounts.spotify.com'
    redirect_uri = os.getenv("REDIRECT_URI")
    
    
    def __init__(self, client_id, client_secret):
        '''
        Initialises the Spotify client with the client ID and secret
        '''
        self.client_id = client_id
        self.client_secret = client_secret
    
    
    def get_client_auth_data(self):
        '''
        Returns the token data necessary for the authorisation post request
        '''
        return {"grant_type": "client_credentials"}            
    
    
    def get_code_auth_data(self):
        '''
        Returns the token data necessary for the authorisation post request
        '''
        return {"grant_type": "authorization_code"}
    
    
    def get_client_auth_headers(self):
        '''
        Returns the token header in base64 encoding necessary for the authorisation post request
        '''
        client_credentials_b64 = self.get_client_credentials()
        return {
            "Authorization":"Basic {credentials}".format(credentials=client_credentials_b64),
            "Content-Type":"application/x-www-form-urlencoded"
        }
    
    
    def get_client_auth_credentials(self):
        ''' 
        Returns a base 64 encoded authorisation string
        '''
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id is None or client_secret is None:
            raise Exception("You must set a client ID and a client secret")
        client_credentials = f'{client_id}:{client_secret}'
        client_credentials_b64 = base64.b64encode(client_credentials.encode())
        return client_credentials_b64.decode()
    
    
    def authorise_client(self):
        '''
        Authorises the client, setting the authorisation token and returning True if successful
        '''
        auth_url = self.auth_url
        auth_data = self.get_client_auth_data()
        auth_headers = self.get_client_auth_headers()
        auth_response = requests.post(auth_url, data=auth_data, headers=auth_headers)
        if auth_response.status_code not in range(200,299):
            raise Exception('Could not authenticate client')
            return False
        auth_response_data = auth_response.json()
        self.access_token = auth_response_data["access_token"]
        expires_in = auth_response_data['expires_in']
        expires = datetime.now() + timedelta(seconds=expires_in)
        self.access_token_expires = expires
        self.access_token_expired = expires < now
        return True
    
    
    def authorise_user(self):
        '''
        Authorises the user
        '''
        endpoint = f'{self.auth_url_user}/authorize'
        query = {
            'client_id':self.client_id, 
            'response_type':'code',
            'scope':'user-top-read',
            'redirect_uri':self.redirect_uri
        }
        user_auth_data = urlencode(query)
        user_auth_url = f'{endpoint}?{user_auth_data}'
        user_auth_response = requests.get(user_auth_url)
        if user_auth_response.status_code not in range (200,299):
            raise Exception('Could not authenticate user')
            return False
        print(user_auth_url)
        print(user_auth_response)
        #self.code = user_auth_response_data['code']
        #self.state = user_auth_response_data['state']
        return user_auth_response.json()
    
    
    def request_tokens(self):
        endpoint = self.auth_url
        query = {
            'grant_type':'authorization_code',
            'code':self.code,
            'redirect_uri':self.redirect_uri
        }
        data = urlencode(query)
        url = f'{endpoint}?{data}'
        headers = self.get_client_auth_headers()
        r = requests.post(url, headers=headers)
        if r.status_code not in range (200,299):
            raise Exception('Could not authenticate client')
            return False
        auth_data = r.json()
        self.access_token = auth_data['access_token']
        self.token_type = auth_data['token_type']
        self.scope = auth_data['scope']
        self.expires_in = auth_data['expires_in']
        self.refresh_token = auth_data['refresh_token']
        return True
        
        
    def get_current_user(self):
        endpoint = f'{self.base_url}/me'
        r = requets.get(endpoint)
        if r.status_code not in rnage(200,299):
            return {}
        return r.json()
    
    
    def get_top_items(self, type='artists'):
        endpoint = f'{self.base_url}/me/{type}'
        r = requests.get(endpoint)
        if r.status_code not in rnage(200,299):
            return {}
        return r.json()
        
        
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.now()
        if expires < now:
            self.authorise()
            return self.access_token
        elif token is None:
            self.authorise()
            return self.access_token 
        return token
            
        
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {'Authorization': f'Bearer {access_token}'}
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums'):
        endpoint = f'{base_url}/{resource_type}/{lookup_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
        
        
    def get_album(self, _id):
        return self.get_resource(_id, 'albums')
    
    
    def get_artist(self, _id):
        return self.get_resource(_id, 'artists')
    
    
    def basic_search(self, query_params):
        endpoint = f'{base_url}/search'
        headers = self.get_resource_header()
        url = f'{endpoint}?{query_params}'
        print(url)
        r = requests.get(url, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
    
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception('A search query is required')
        if isinstance(query, dict):
            query = ' '.join([f'{k}:{v}' for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == 'or' or operator.lower() == 'not':
                operator = operator.upper()
                if isinstace(operator_query, str):
                    query = f'{query} {operator} {operator_query}'
        query_params = urlencode({'q': query, 'type': search_type.lower()})
        print(query_params)
        return self.basic_search(query_params)
    
    
    def get_user(self, _id):
        return self.get_resource(_id, 'users')
    
    
    def get_user_saved_tracks(self, _id):
        return self.get_resource(_id, 'me/tracks')
    
    
    def get_current_user(self, resource_type='authorize'):
        endpoint = f'{self.base_url}/{resource_type}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        print(r)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
        

In [32]:
spotify = SpotifyAPI(client_id, client_secret)

In [33]:
spotify.authorise_user()

https://accounts.spotify.com/authorize?client_id=6e1ae3a813ca4ae2ba228993b7b3e229&response_type=code&scope=user-top-read&redirect_uri=http%3A%2F%2Flocalhost%3A8501
<Response [200]>


JSONDecodeError: [Errno Expecting value] 
<!DOCTYPE html>
<html id="app" lang="en" dir="ltr">
<head>
  <meta charset="utf-8">
  <title>Login - Spotify</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
  <base href="/">
  <link rel="icon" href="https://accounts.scdn.co/sso/images/favicon.ace4d8543bbb017893402a1e9d1ac1fa.ico">

    <link rel="preload" href="https://encore.scdn.co/1.2.3/CircularSpotify-UI-Latin-OS2v3-Light.woff2" as="font" type="font/woff2" crossOrigin="true"/>
    <link rel="preload" href="https://encore.scdn.co/1.2.3/CircularSpotify-UI-Latin-OS2v3-Book.woff2" as="font" type="font/woff2" crossOrigin="true"/>
    <link rel="preload" href="https://encore.scdn.co/1.2.3/CircularSpotify-UI-Latin-OS2v3-Bold.woff2" as="font" type="font/woff2" crossOrigin="true"/>
    <link rel="preload" href="https://encore.scdn.co/1.2.3/CircularSpotify-UI-Latin-OS2v3-Black.woff2" as="font" type="font/woff2" crossOrigin="true"/

In [17]:
print(spotify.redirect_uri)

http://localhost:8501


In [9]:
spotify.get_current_user()

AttributeError: 'SpotifyAPI' object has no attribute 'access_token_expires'

In [ ]:
'''
TODO
Need to figure out how to decode/ unquote the query parameters returned from the 
OAuth validation to be sent through to the get request token method
'''